In [24]:
from numpy import *
import numpy as np
import time
from copy import copy, deepcopy
import pandas as pd

In [35]:
def dsig(value, sig_digits):
    if value == 0:
        return 0
    return round(value, sig_digits - int(math.floor(math.log10(abs(value)))) - 1)

In [ ]:
#generate test data

In [27]:
test_data_range=1100

In [28]:
def generate_test_data(test_range):
    test_cases = {}
    for case in range(100, test_range, 100):
        matrix_A = [[dsig(random.random(), sig_digits=5) for _ in range(case)]for _ in range(case)]
        matrix_b = [dsig(random.random(), sig_digits=5) for _ in range(case)]
        test_cases[case] = {'matrix_A': matrix_A, 'matrix_b': matrix_b}

    return test_cases

In [29]:
test_suit = generate_test_data(test_data_range)
for each_case in test_suit.items():
    print(each_case[0])
    print(f"matrix_A: {len(each_case[1]['matrix_A'])} * {len(each_case[1]['matrix_A'][0])}")
    print(f"matrix_b: {len(each_case[1]['matrix_b'])}")
    print("-"*30)

100
matrix_A: 100 * 100
matrix_b: 100
------------------------------
200
matrix_A: 200 * 200
matrix_b: 200
------------------------------
300
matrix_A: 300 * 300
matrix_b: 300
------------------------------
400
matrix_A: 400 * 400
matrix_b: 400
------------------------------
500
matrix_A: 500 * 500
matrix_b: 500
------------------------------
600
matrix_A: 600 * 600
matrix_b: 600
------------------------------
700
matrix_A: 700 * 700
matrix_b: 700
------------------------------
800
matrix_A: 800 * 800
matrix_b: 800
------------------------------
900
matrix_A: 900 * 900
matrix_b: 900
------------------------------
1000
matrix_A: 1000 * 1000
matrix_b: 1000
------------------------------


In [ ]:
#without pivoting

In [30]:
def forward_elimination(A,b,n,digit):
    #forward elimination
    countA,countD,rowsum=0,0,0
    for k in range (0,n-1):             #row is k
        for i in range(k+1,n):
            if(A[k,k]!=0):
                factor=A[i,k] / A[k,k]
                countD+=1
            A[i][k]=0
            for j in range(k+1,n):
                if(A[i,j]==0):
                    j+1
                if(A[i,j]!=0):
                    A[i,j]=A[i,j]-(factor*A[k,j])
                    countA+=1
            b[i]=b[i]-(factor*b[k])
            countA+=1
    operation_count = {
        'addition': countA,
        'multiplication': countA,
        'division': countD
    }       
#     print ("Forward Elimination A \n",A,"\n Forward Elimination b \n",b,"\n")
    return operation_count

In [21]:
def backward_sub(A,b,n,digit):
    cntA,cntD,row_sum=0,0,0
    x=np.zeros(n)
    x[n-1] = b[n-1]/A[n-1,n-1]
    cntD+=1
    for row in range(n-2,-1,-1):
        sum = b[row]
        for j in range(row+1,n):
            sum=sum - (A[row,j] * x[j])
            cntA+=1
        x[row]=sum/A[row,row]
        cntD+=1
        operation_cnt = {
        'addition': cntA,
        'multiplication': cntA,
        'division': cntD
    }    
#     print ("Backward Elimination A \n",A,"\n Backward Elimination b \n",b,"\n")
    return operation_cnt

In [22]:
def gauss_without_pivoting(A,b,digit):
    A = np.array(A)
    b = np.array(b)
    matrixA = np.matrix.copy(A)
    matrixb = np.matrix.copy(b)
    n=len(b)
    forward_count = forward_elimination(A,b,n,digit)
    backward_count=backward_sub(matrixA,matrixb,n,digit)
    out = {k: forward_count[k] + backward_count[k] for k in forward_count.keys() & backward_count.keys()}
    return out

In [23]:
for case_key, each_case in test_suit.items():
    print(case_key)
    operation = gauss_without_pivoting(A = deepcopy(each_case['matrix_A']), b = deepcopy(each_case['matrix_b']), digit=5)
    each_case['opcount'] = operation
    print(each_case['opcount'])
    print("+"*30, "\n\n")

3
{'multiplication': 11, 'addition': 11, 'division': 6}
++++++++++++++++++++++++++++++ 




In [ ]:
#with_pivoting

In [44]:
def forward_with_pivoting(A,b,n,digit):   
    countA,countD,rowsum=0,0,0
    for k in range (0,n-1): #row is k
        for i in range (k+1,n):
            if (abs(A[i][k]) > abs(A[k][k])):
                temp = np.copy(A[k])
                A[k] = A[i]
                A[i] = temp
            factor=dsig((A[i,k] / A[k,k]),digit)
            countD+=1
            A[i][k]=0
            for j in range(k+1,n):
                if(A[i,j]==0):
                    j+1
                if(A[k,j]!=0):
                    A[i,j]=dsig((A[i,j]-dsig((factor*A[k,j]),digit)),digit)
                    countA+=1
            b[i]=dsig((b[i]-dsig((factor*b[k]),digit)),digit)
            countA+=1
    operation_count = {
        'addition': countA,
        'multiplication': countA,
        'division': countD
    }       
#     print ("Forward Elimination A \n",A,"\n Forward Elimination b \n",b,"\n")
    return operation_count

In [45]:
def backward_with_pivoting(A,b,n,digit):
    cntA,cntD,row_sum=0,0,0
    x=np.zeros(n)
    x[n-1] = b[n-1]/A[n-1,n-1]
    cntD+=1
    for row in range(n-2,-1,-1):
        sum = b[row]
        for j in range(row+1,n):
                sum=sum - (A[row,j] * x[j])
                cntA+=1
        x[row]=sum/A[row,row]
        cntD+=1
        operation_cnt = {
        'addition': cntA,
        'multiplication': cntA,
        'division': cntD
    }    
#     print ("Backward Elimination A \n",A,"\n Backward Elimination b \n",b,"\n")
    return operation_cnt

In [46]:
def gauss_with_pivoting(A,b,digit):
    A = np.array(A)
    b = np.array(b)
    matrixA = np.matrix.copy(A)
    matrixb = np.matrix.copy(b)
    n=len(b)
    sol = np.zeros(n)
    sol = np.zeros(n)
    forward_count=forward_with_pivoting(A,b,n,digit)
    backward_count=backward_with_pivoting(matrixA,matrixb,n,digit)
    out = {k: forward_count[k] + backward_count[k] for k in forward_count.keys() & backward_count.keys()}
    return out

In [47]:
for case_key, each_case in test_suit.items():
    print(case_key)
    oper = gauss_with_pivoting(A = deepcopy(each_case['matrix_A']), b = deepcopy(each_case['matrix_b']), digit=5)
    each_case['opcount_with_pivoting'] = oper
    print(each_case['opcount_with_pivoting'])
    print("+"*30, "\n\n")

100
{'multiplication': 338250, 'addition': 338250, 'division': 5050}
++++++++++++++++++++++++++++++ 


200


KeyboardInterrupt: 